In [287]:
p = 58790796
hexdump.hexdump(c[p:p+192])

00000000: 5B 02 61 00 01 00 00 00  0F 00 00 00 50 28 0F 3C  [.a.........P(.<
00000010: 46 32 1E 14 78 64 0A 05  23 19 03 00 00 00 00 00  F2..xd..#.......
00000020: 00 00 00 00 00 00 00 00  00 00 00 00 57 BD 39 00  ............W.9.
00000030: 2A 00 00 00 20 00 00 00  3C 00 00 00 7C 05 00 00  *... ...<...|...
00000040: 6C 00 00 00 68 3E 01 00  0D 86 00 00 1C 34 B9 00  l...h>.......4..
00000050: 72 B3 30 00 40 28 BA 00  3D 31 30 00 89 8B 00 00  r.0.@(..=10.....
00000060: 6C 00 00 00 5C 7E 02 00  7A 05 01 00 C4 95 B9 00  l...\~..z.......
00000070: 3D 31 30 00 40 28 BA 00  07 AF 2F 00 03 91 01 00  =10.@(..../.....
00000080: 6C 00 00 00 24 B1 00 00  D7 49 00 00 1C 34 B9 00  l...$....I...4..
00000090: 3D 31 30 00 C4 95 B9 00  22 F0 2F 00 DA DA 01 00  =10....."./.....
000000A0: 6C 00 00 00 B0 BE 01 00  88 B7 00 00 F0 64 B9 00  l............d..
000000B0: 22 F0 2F 00 C4 95 B9 00  95 CF 2F 00 62 92 02 00  "./......./.b...


In [400]:
c.find(b'\x5b\x02\x61\x00')
list(find_all(c, b'\x5b\x02\x61\x00'))
dict(zip(range(5), range(5))), (1, 2) == (1, 3)

({0: 0, 1: 1, 2: 2, 3: 3, 4: 4}, False)

In [618]:
import struct
import inspect
import json

class RP():
    def __init__(self, filename='/tmp/9.rp'):
        content = open(filename, 'rb').read()
        self.content = content
        c = content
        self.f_2, = struct.unpack('i', c[8:12]) ## file_size
        self.f_1, = struct.unpack('i', c[4:8])  ## header size
        self.f_15, = struct.unpack('i', c[60:64])
        self.f_16, = struct.unpack('i', c[64:68])
        self.f_17, = struct.unpack('i', c[68:72])
        self.f_18, = struct.unpack('i', c[72:76])
        self.f_19, = struct.unpack('i', c[76:80])  ## road names begin
        self.f_20, = struct.unpack('i', c[80:84])  ## road names end, DistrictFrame begin
        self.f_21, = struct.unpack('i', c[84:88])
        self.f_22, = struct.unpack('i', c[88:92])
        self.f_23, = struct.unpack('i', c[92:96])
        self.levels = []
        for i in range(self.f_21):
            vals = struct.unpack('Iiiiiiii', c[self.f_23+self.f_22*i:self.f_23+self.f_22*i+self.f_22])
            keys = ('a_0', 'CalcRegionManagerHeader_pos_1', 'na_2', 'CalcRegionManagerHeader_len_3', 
                    'InfoRegionManagerHeader_pos_4', 'InfoRegionManagerHeader_len_5', 'AdjacentNodeManager_pos_6', 
                    'AdjacentNodeManager_len_7')
            self.levels.append(dict(zip(keys, vals)))
        pass
        ## the name frame part
        name_frame_size, header_size, item_cnt, idx_item_size, header_size_2, names_offset = struct.unpack('iiiiii', c[self.f_19:self.f_19+24])
        assert (header_size, idx_item_size, header_size_2, self.f_19+name_frame_size) == (24, 4, 24, self.f_20)
        all_name = c[self.f_19+names_offset:self.f_20].decode('utf-16')
        names = []
        p = self.f_19+header_size
        for i in range(item_cnt):
            item_len, item_pos = struct.unpack('<bi', c[p:p+4]+b'\x00')
            names.append(all_name[item_pos:item_pos+item_len])
            p += 4
        self.names = names
        ## the info region part
        self.info_region_manager_headers = []
        for level in self.levels:
            header = struct.unpack('iiiiiiiiiiiiiii', c[level['InfoRegionManagerHeader_pos_4']:level['InfoRegionManagerHeader_pos_4']+60])
            zipped_end, region_cnt, idx_item_size = header[11:14]
            assert idx_item_size == 32
            l = {}
            l['zipped_end'], l['region_cnt'], l['regions'] = zipped_end, region_cnt, []
            p =level['InfoRegionManagerHeader_pos_4']
            for i in range(region_cnt):
                data_offset, _, unzipped_len, zipped_len = struct.unpack('iiii', c[p+60+32*i:p+60+32*i+16])
                l['regions'].append((p+data_offset, p+data_offset+zipped_len, unzipped_len))
            self.info_region_manager_headers.append(l)
    def get_info_region_buff(self, level, idx):
        begin, end, unzipped_size = self.info_region_manager_headers[level]['regions'][idx]
        ret = zlib.decompress(self.content[begin:end])
        assert len(ret) == unzipped_size
        return ret
    def __repr__(self):
        atts = {}
        for i in dir(self):
#             print(i)
            if i[0] == '_' or i in ('contentaa', ) :
                continue
            att = self.__getattribute__(i)
            if type(att) == bytes:
                continue
            if inspect.ismethod(att):
                continue
            atts[i] = att
#         print(type(atts))
#         print(atts.keys())
#         return 'gogogo'
        return '%s ==> %s'%('hello',atts.__repr__())

rp = RP()

In [544]:
rp.info_region_manager_headers[0]

{'region_cnt': 299,
 'regions': [(28907905, 28989152, 191684),
  (28989152, 29047430, 140696),
  (29047430, 29071554, 54668),
  (29071554, 29159695, 210584),
  (29159695, 29249677, 222028),
  (29249677, 29297650, 118500),
  (29297650, 29330495, 80036),
  (29330495, 29461916, 323776),
  (29461916, 29474148, 27992),
  (29474148, 29590457, 280164),
  (29590457, 29647424, 133044),
  (29647424, 29773576, 310464),
  (29773576, 29791237, 39816),
  (29791237, 29860759, 163756),
  (29860759, 29940072, 189984),
  (29940072, 29986979, 113312),
  (29986979, 30124316, 346348),
  (30124316, 30203913, 190732),
  (30203913, 30273654, 167076),
  (30273654, 30284795, 25116),
  (30284795, 30382242, 240172),
  (30382242, 30507704, 310840),
  (30507704, 30507844, 240),
  (30507844, 30648095, 339668),
  (30648095, 30709875, 156260),
  (30709875, 30743223, 80752),
  (30743223, 30847800, 265892),
  (30847800, 30907698, 153516),
  (30907698, 30981155, 188284),
  (30981155, 31054311, 181640),
  (31054311, 31171

In [320]:
def buff_2_ints(buff):
    ret = []
    while buff:
        i, buff = buff[:4], buff[4:]
        item, = struct.unpack('i', i)
        ret.append(item)
    return ret

def buff_2_int16s(buff):
    ret = []
    while buff:
        i, buff = buff[:2], buff[2:]
        item, = struct.unpack('h', i)
        ret.append(item)
    return ret

In [299]:
p = 192   ## CalcRegionManagerHeader
(buff_2_ints(c[p:p+20]), 299*16 + 20, buff_2_ints(c[p+20:p+20+16]), buff_2_ints(c[p+20+16:p+20+32]), 
buff_2_ints(c[p+20+32:p+20+48]))

([28898085, 0, 299, 16, 20],
 4804,
 [4804, 4804, 224704, 93599],
 [98403, 229508, 147556, 61624],
 [160027, 377064, 71288, 29179])

In [424]:
p = 28898277 ## AdjacentNodeManager
buff_2_ints(c[p:p+60]), 32*299+60

([25678353,
  1,
  15,
  1007626320,
  337523270,
  84567160,
  203043,
  0,
  0,
  0,
  0,
  23104581,
  299,
  32,
  60],
 9628)

In [430]:
p = 58790796 ## InfoRegionManagerHeader
buff_2_ints(c[p:p+60]), 42*32+60

([6357595,
  1,
  15,
  1007626320,
  337523270,
  84567160,
  203043,
  0,
  0,
  0,
  0,
  3784023,
  42,
  32,
  60],
 1404)

In [393]:
a = b'\xa7\x63\x5f\x6c\xef\x8d'
a.decode('utf-16')
len(rp.content)
# rp.content.find(a)
rp

hello ==> {'f_1': 192, 'f_15': 1140252, 'f_16': 1168420, 'f_17': 178, 'f_18': 19, 'f_19': 54576630, 'f_2': 68245031, 'f_20': 54774396, 'f_21': 3, 'f_22': 32, 'f_23': 96, 'levels': [{'a_0': 3221225472, 'CalcRegionManagerHeader_pos_1': 192, 'na_2': 192, 'CalcRegionManagerHeader_len_3': 4804, 'InfoRegionManagerHeader_pos_4': 28898277, 'InfoRegionManagerHeader_len_5': 9628, 'AdjacentNodeManager_pos_6': 54774488, 'AdjacentNodeManager_len_7': 25620}, {'a_0': 3271557120, 'CalcRegionManagerHeader_pos_1': 54800108, 'na_2': 126304530, 'CalcRegionManagerHeader_len_3': 692, 'InfoRegionManagerHeader_pos_4': 58790796, 'InfoRegionManagerHeader_len_5': 1404, 'AdjacentNodeManager_pos_6': 65148391, 'AdjacentNodeManager_len_7': 4020}, {'a_0': 3238002688, 'CalcRegionManagerHeader_pos_1': 65152411, 'na_2': 147070066, 'CalcRegionManagerHeader_len_3': 84, 'InfoRegionManagerHeader_pos_4': 65418745, 'InfoRegionManagerHeader_len_5': 188, 'AdjacentNodeManager_pos_6': 68244115, 'AdjacentNodeManager_len_7': 916}]}

In [405]:
p = 54576630
buff_2_ints(c[p:p+24]), struct.unpack('bbbbbbbb',c[54635510-8:54635510])

([197766, 24, 14714, 4, 24, 58880], (2, 60, 15, 1, 3, 63, 15, 1))

In [398]:
14714*4, 54576630+58880

(58856, 54635510)

In [50]:
b = '南北高架'
b = b.encode('utf-16')
c.find(b[2:])

54704544

In [20]:
c = rp.content
p = 54692560
p = 54692560
c[p:p+100]

b'\xa7c_l\xef\x8d\x00\x00\xa7c_l\xef\x8d\x85\x8f\xef\x8d\x00\x00\xe8\x8d\x99\x9f\xef\x8d\x00\x00\xe8\x8d\x16Y\xafs\xbf~eh\x00\x00\xeb_PN\x17S\xef\x8d\x00\x00\xeb_PNWS-N\xc3_\xef\x8d\x00\x00!S\xf7]\xef\x8d\x00\x00NYgq\xef\x8d\x00\x00dW\xb2\x80\xef\x8d\x00\x00\x06f\x17S/e\xef\x8d\x00\x00\x06f'

In [101]:
struct.unpack('iiiiii', c[54576630:54576630+24])

(197766, 24, 14714, 4, 24, 58880)

In [278]:
p = 52002858
struct.unpack('ii', c[p:p+8]), 584025, 643392*4, 52002858+643442*4

((643442, 68), 584025, 2573568, 54576626)

In [270]:
c[54774396:54774396+92]

b'\\\x00\x00\x00\x14\x00\x00\x00\x12\x00\x00\x00\x04\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [103]:
aa = []
for i in range(14714):
    j, = struct.unpack('i', c[54576630+24+i*4:54576630+24+i*4+4])
    aa.append(j)

In [118]:
aa[:10]

[8, 2310, 4102, 5894, 7686, 9476, 10758, 12550, 14341, 15878]

In [100]:
54635510 - 54576630, 14714*4
# 197766 + 54576630


(58880, 58856)

In [419]:
rp.names[-10:]

['沪蓉高速', '中国航海博物馆', '两港大道', '周氏立交', '东环中线', '马陆', '佘山', '杭州湾大桥', '张庄', '民星路']

In [267]:
names = c[54635510:54774396].decode('utf-16')
names = names.split('\x00')

In [268]:
len(names)
# names[916]

14715

In [131]:
c[189+92481+96518+ 64764:].find(b'\x94\x03')
b = 'abc'
b.f

In [133]:
pp = 62574819
struct.unpack('i', c[pp:pp+4]), 0x9d140

((643392,), 643392)

In [167]:
dd = open('/tmp/zip.bin', 'rb').read()
udd = zlib.decompress(dd+b'aaaaa')
len(udd)
zdd = zlib.compress(udd)
len(zdd)

148409

In [169]:
# zlib.decompress(c[31833528:])

In [237]:
ppp = 31833528
p1 = 28899597

block_b, block_e = 0, 0

for i in range(1000):
#     zipped_len, = struct.unpack('i', c[p1-32:p1-32+4])
    zipped_len, = struct.unpack('i', c[p1:p1+4])
    assert c[ppp:ppp+2] == b'\x78\x9c'
#     print('...')
    block_b, block_e = ppp, ppp+zipped_len
    p1 += 32
    ppp += zipped_len

# for i in range(10):
#     udd = zlib.decompress(c[ppp:])
#     unzipped_size = len(udd)
#     zdd = zlib.compress(udd)
#     zipped_size = len(zdd)
#     print(unzipped_size, zipped_size)
#     ppp += zipped_size

AssertionError: 

In [197]:
block_b, block_e, p1

(28907905, 28907920, 28898285)

In [201]:
block_b, block_e, p1

(52002820, 52002858, 28907917)

In [143]:
import hexdump, zlib
c.find(dd)

31833528

In [236]:
u1 = zlib.decompress(c[58826517:])
u2 = zlib.compress(u1)
hex(len(u1)), hex(len(u2))

('0x27e5c', '0x1057a')

In [215]:
p = 28907905
for i in range(10000):
    if c[p:p+32].find(b'\xc4\xec\x02\x00\x5f\x3d\x01') >= 0:
        print(p)
        break
    p -= 32

28898337


In [205]:
hex(52002858-52002820), hex(28907920-28907905)

('0x26', '0xf')

In [235]:
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub) # use start += 1 to find overlapping matches

list(find_all(c, b'\xb9C\x02\x00')), (list(find_all(c, b'\x78\x9c\x7c\x7d')))

([28899597],
 [28907905,
  29047430,
  29297650,
  29330495,
  29590457,
  29940072,
  29986979,
  30124316,
  30203913,
  30284795,
  30507844,
  30981155,
  31054311,
  31171031,
  31253411,
  31371090,
  31518118,
  31616772,
  31681807,
  31833528,
  31981937,
  32075752,
  32196429,
  32735277,
  32805951,
  32863168,
  33346993,
  33442871,
  33559159,
  33671678,
  33853818,
  34120740,
  34394497,
  34509092,
  34599680,
  34659674,
  34807950,
  34915102,
  35240048,
  35599557,
  35696526,
  35935049,
  36837928,
  37018959,
  37280038,
  37383575,
  37484622,
  37558789,
  37645004,
  37780396,
  37947352,
  38040487,
  38176006,
  38230007,
  38296795,
  38353451,
  38477530,
  38545644,
  38794920,
  38831946,
  38883945,
  39465352,
  39534770,
  39617473,
  39744312,
  39816272,
  40097649,
  40182808,
  40256918,
  40430468,
  40604669,
  40680858,
  40728057,
  40775397,
  40906043,
  41020865,
  41123059,
  41243139,
  41335837,
  41460640,
  41671792,
  41783383,
  4

In [319]:
list(find_all(c, b'\x78\xb3\x02\x00'))

[3244, 28899945]

In [179]:
print(struct.pack('i', 369876))
(hex(ppp), list(find_all(c, struct.pack('i', 148409))), 
 list(find_all(c, struct.pack('i', 44944))), list(find_all(c, struct.pack('i', 48871))),
    list(find_all(c, struct.pack('i', 120677))))28898277

b'\xd4\xa4\x05\x00'


('0x1f1c509', [28899597], [28899629], [28899661], [28899693])

In [264]:
28899693-28899661, 28899661-28899629, 28899629-28899597, 0x3c

(32, 32, 32, 60)

In [247]:
28898277+0x259c, 28898337-28898277, 28907905 - 28898337 - 0x259c

(28907905, 60, -60)

In [226]:
(28907905 - 28898337)/32, 28898337-28898277, hex(299), hex(0x162fb-0x259c), hex(0x246a1-0x162fb), hex(28907905 - 28898337)

(299.0, 60, '0x12b', '0x13d5f', '0xe3a6', '0x2560')

In [266]:
def ana_from_arr_begin(p):
    rows, = struct.unpack('i', c[p-16+4:p-16+8])
    zipped_begin = p + 32*rows
    zipped = zipped_begin
    for i in range(rows):
        unzipped_size, zipped_size = struct.unpack('ii', c[p+8:p+16])
        ud = zlib.decompress(c[zipped:zipped+zipped_size])
        assert len(ud) == unzipped_size
        p += 32
        zipped += zipped_size
    print('%d zipped records begin:%d end:%d'%(rows, zipped_begin, zipped))

ana_from_arr_begin(28898329+8), ana_from_arr_begin(58790848+8), ana_from_arr_begin(65418797+8)

299 zipped records begin:28907905 end:52002858
42 zipped records begin:58792200 end:62574819
4 zipped records begin:65418933 end:65672047


(None, None, None)

In [434]:
ppp = 28898277+60  ##28898337
# ppp += 148409
hexdump.hexdump(c[ppp:ppp+0x100])
len(c)

00000000: 9C 25 00 00 6C 00 00 00  C4 EC 02 00 5F 3D 01 00  .%..l......._=..
00000010: C4 95 B9 00 72 B3 30 00  40 28 BA 00 3D 31 30 00  ....r.0.@(..=10.
00000020: FB 62 01 00 6C 00 00 00  98 25 02 00 A6 E3 00 00  .b..l....%......
00000030: 1C 34 B9 00 72 B3 30 00  C4 95 B9 00 57 72 30 00  .4..r.0.....Wr0.
00000040: A1 46 02 00 6C 00 00 00  8C D5 00 00 3C 5E 00 00  .F..l.......<^..
00000050: F0 64 B9 00 57 72 30 00  C4 95 B9 00 CA 51 30 00  .d..Wr0......Q0.
00000060: DD A4 02 00 6C 00 00 00  98 36 03 00 4D 58 01 00  ....l....6..MX..
00000070: 1C 34 B9 00 57 72 30 00  F0 64 B9 00 CA 51 30 00  .4..Wr0..d...Q0.
00000080: 2A FD 03 00 6C 00 00 00  4C 63 03 00 7E 5F 01 00  *...l...Lc..~_..
00000090: F0 64 B9 00 CA 51 30 00  C4 95 B9 00 3D 31 30 00  .d...Q0.....=10.
000000A0: A8 5C 05 00 6C 00 00 00  E4 CE 01 00 65 BB 00 00  .\..l.......e...
000000B0: 86 4C B9 00 84 41 30 00  F0 64 B9 00 3D 31 30 00  .L...A0..d..=10.
000000C0: 0D 18 06 00 6C 00 00 00  A4 38 01 00 4D 80 00 00  ....l....8..M...

68245031

In [428]:
ppp = 58790796  ##28898337
# ppp += 148409
hexdump.hexdump(c[ppp:ppp+0x100])

00000000: 5B 02 61 00 01 00 00 00  0F 00 00 00 50 28 0F 3C  [.a.........P(.<
00000010: 46 32 1E 14 78 64 0A 05  23 19 03 00 00 00 00 00  F2..xd..#.......
00000020: 00 00 00 00 00 00 00 00  00 00 00 00 57 BD 39 00  ............W.9.
00000030: 2A 00 00 00 20 00 00 00  3C 00 00 00 7C 05 00 00  *... ...<...|...
00000040: 6C 00 00 00 68 3E 01 00  0D 86 00 00 1C 34 B9 00  l...h>.......4..
00000050: 72 B3 30 00 40 28 BA 00  3D 31 30 00 89 8B 00 00  r.0.@(..=10.....
00000060: 6C 00 00 00 5C 7E 02 00  7A 05 01 00 C4 95 B9 00  l...\~..z.......
00000070: 3D 31 30 00 40 28 BA 00  07 AF 2F 00 03 91 01 00  =10.@(..../.....
00000080: 6C 00 00 00 24 B1 00 00  D7 49 00 00 1C 34 B9 00  l...$....I...4..
00000090: 3D 31 30 00 C4 95 B9 00  22 F0 2F 00 DA DA 01 00  =10....."./.....
000000A0: 6C 00 00 00 B0 BE 01 00  88 B7 00 00 F0 64 B9 00  l............d..
000000B0: 22 F0 2F 00 C4 95 B9 00  95 CF 2F 00 62 92 02 00  "./......./.b...
000000C0: 6C 00 00 00 64 8E 04 00  1C DD 01 00 1C 34 B9 00  l...d........4..

In [261]:
ppp = 65418933  ##28898337

# ppp = 58826517

# ppp = 28898337  ##28898337
# ppp += 148409
hexdump.hexdump(c[ppp-0x150:ppp+0x1000])

00000000: D8 85 51 99 89 1B 79 AF  99 2A B6 E3 47 D6 61 32  ..Q...y..*..G.a2
00000010: 1E E2 54 3E 64 2F 16 63  33 D6 E5 5F 76 E5 46 C6  ..T>d/.c3.._v.F.
00000020: E4 51 DE E5 38 1E 64 A7  E6 DC A3 3B 8D 0F 79 8E  .Q..8.d....;..y.
00000030: FD B8 94 5D D9 6C 88 97  D9 97 8F 39 6E 8C 1F 79  ...].l.....9n..y
00000040: 92 DD D6 F8 91 27 D9 ED  1B 3F F2 24 CF 71 32 C7  .....'...?.$.q2.
00000050: DE F1 24 07 73 36 A7 33  F9 1C 3F 2E B1 F5 14 0F  ..$.s6.3..?.....
00000060: 32 DC 15 33 F2 24 3F 72  20 5F 32 1F 53 33 22 C3  2..3.$?r _2.S3".
00000070: 32 22 F3 F2 2D CF 7A C0  19 EB B2 26 E3 F3 21 2F  2"..-.z....&..!/
00000080: 32 3B 7F FB 01 23 32 32  2B F3 3B 9B F3 3C E7 BF  2;...#22+.;..<..
00000090: FB 41 3E 50 9A 1C 2B 00  01 00 00 00 0F 00 00 00  .A>P..+.........
000000A0: 50 28 0F 3C 46 32 1E 14  78 64 0A 05 23 19 03 00  P(.<F2..xd..#...
000000B0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
000000C0: 76 DD 03 00 04 00 00 00  20 00 00 00 3C 00 00 00  v....... ...<...

In [203]:
ppp = 28907905
# ppp += 148409
hexdump.hexdump(c[ppp-0x40:ppp+0x200])

00000000: F9 8B 60 01 6C 00 00 00  6C 00 00 00 26 00 00 00  ..`.l...l...&...
00000010: A0 A1 B8 00 B7 EB 2E 00  74 D2 B8 00 2A CB 2E 00  ........t...*...
00000020: 1F 8C 60 01 6C 00 00 00  6C 00 00 00 26 00 00 00  ..`.l...l...&...
00000030: F8 3F B8 00 D2 2C 2F 00  A0 A1 B8 00 2A CB 2E 00  .?...,/.....*...
00000040: 78 9C 7C 7D 0B 9C 55 53  FB FF B3 F7 39 73 3A 33  x.|}..US....9s:3
00000050: 0E 9D 33 73 AA BD 67 EF  38 33 4D 35 A5 CB 94 AE  ..3s..g.83M5....
00000060: 84 D3 3D 09 53 42 28 A6  4C 09 95 29 45 14 A6 69  ..=.SB(.L..)E..i
00000070: AA 89 30 25 84 30 25 84  10 E2 0D 61 74 51 11 42  ..0%.0%....atQ.B
00000080: 14 85 10 72 7D 73 BF E5  F5 5F FB 59 CF DA 7B ED  ...r}s..._.Y..{.
00000090: BD F6 EF FF BE 1F 6F BF  1F 5F CF 5A CF B3 9E DB  ......o.._.Z....
000000A0: FA AE B5 D7 01 00 E8 99  77 39 CC 62 7F 5E 56 04  ........w9.b.^V.
000000B0: F0 60 1E C0 04 ED 48 98  09 37 C3 3E 88 B2 FF 02  .`....H..7.>....
000000C0: 94 42 12 FF 2F E7 7F 4B  21 CD FE 37 0E 06 FB EF  .B../..K!..7....

In [636]:
abc = [1, 2, 3]
# abc.remove(1)
del abc[1]
abc[:-1]

# iii = [(0, 1), (1, 4), (4,9), (9,16), (11, 12), (12, 13), (13, 11)]
# import random
# random.shuffle(iii)
# iii

[1]

In [654]:
iii = [(12, 13), (11, 12), (13, 11), (4, 9), (1, 4), (0, 1), (9, 16), (20, 21, 22), (22, 25), (25, 20)]
combine_links_2(iii)

[(13, 11, 12, 13), (0, 1, 4, 9, 16), (25, 20, 21, 22, 25)]

In [610]:
rp.names.index('内江路')

7589

In [580]:
uu = get_road_points(rp)

In [621]:
rp.names[2301], len(uu)

('东体育会支路', 14140)

In [633]:
len(uu[10914])

145

In [634]:
combine_links(uu[10914])

KeyboardInterrupt: 

In [664]:
dump_road_points(rp, level=2)

In [661]:
def combine_links(links):
    tails = {}
    heads = {}
    ls = links[:]
    for l in ls:
#         print('----------------')
#         print(l)
#         print(heads)
#         print(tails)
        p = l[:]
        if p[-1] in heads and heads[p[-1]]:
            p = p + heads[p[-1]][0][1:]
            del heads[l[-1]][0]
            del tails[p[-1]]
        if p[0] in tails and tails[p[0]]:
            p = tails[p[0]][0][:-1] + p
            del tails[l[0]][0]
            del heads[p[0]]
        if p[0] not in heads:
            heads[p[0]] = []
        heads[p[0]].append(p)
        if p[-1] not in tails:
            tails[p[-1]] = []
        tails[p[-1]].append(p)
    ret = []
    for i in heads.values():
        ret += i
    return ret

def get_road_points(rp, level=0):
    buckets = {}
    ret = {}
    for idx in range(rp.info_region_manager_headers[level]['region_cnt']):
        buff = rp.get_info_region_buff(level, idx)
        ibf = InfoRegionBuffer(buff)
        for idx_0 in range(ibf.row_count[0]):
            name_idx, link_idx = ibf.get_named_link_info(idx_0)
            ps = ibf.get_path_points(link_idx)
            if name_idx not in buckets:
                buckets[name_idx] = []
            buckets[name_idx].append(ps)
    return buckets

def dump_road_points(rp, file_path='/tmp/rp_9.txt', points=None, level=0):
    if not points:
        points = get_road_points(rp, level)
    f = open(file_path, 'w')
#     n = 0
    for name in points:
#         if rp.names[name] == '无名路':
#             continue
#         print('dumping', name, rp.names[name])
        row = {'road_name':rp.names[name], 'links':combine_links(points[name])}
        f.write(json.dumps(row, ensure_ascii=False))
        f.write('\n')
#         n += 1
#         if n%100 == 0:
#             print('dumped %d lines'%n)
    f.close()
            
class InfoRegionBuffer():
    def __init__(self, raw):
        self.data = raw
        self.offsets = struct.unpack('iiiiiiiiiiii', raw[60:108])
        self.row_size = struct.unpack('hhhhhhhhhhhh', raw[36:60])
        self.row_count = struct.unpack('hhiiihhhhhhi', raw[4:36])
        assert self.row_size == (40, 16, 4, 4, 16, 4, 40, 20, 4, 8, 24, 24)
        pos = 108
        for i in range(12):
#             print(self.offsets[i], pos)
            if self.row_count[i]:
                assert self.offsets[i] == pos
            pos += self.row_count[i]*self.row_size[i]
    def get_path_points(self, idx):
        x, y, n, offset = struct.unpack('iiii', self.data[self.offsets[4]+idx*16:self.offsets[4]+idx*16+16])
        ret = [(x, y)]
        for i in range (n):
            dx, dy = struct.unpack('hh', self.data[self.offsets[3]+(offset+i)*4:self.offsets[3]+(offset+i+1)*4])
            x, y = x+dx, y+dy
            ret.append((x, y))
        return ret
    def get_named_link_points(self, idx):
        t = struct.unpack('iiiiiiiiii', self.data[self.offsets[0]+idx*self.row_size[0]:self.offsets[0]+(idx+1)*self.row_size[0]])
        name_idx, link_idx = t[1], t[7]
        return name_idx, self.get_path_points(link_idx)
    def get_named_link_info(self, idx):
        t = struct.unpack('iiiiiiiiii', self.data[self.offsets[0]+idx*self.row_size[0]:self.offsets[0]+(idx+1)*self.row_size[0]])
        name_idx, link_idx = t[1], t[7]
        return name_idx, link_idx
    def some_test(self):
        p = 108
        row_size = self.row_size[0]
        ids = set()
        for i in range(100):
            row = buff_2_ints(self.data[p+row_size*i:p+row_size*(i+1)])
            ids.add(row[1])
        return ids
    def t2(self):
        p = self.offsets[4]
        row_size = self.row_size[4]
        size = self.row_count[4]
        for i in range(size):
            row = buff_2_ints(self.data[p+row_size*i:p+row_size*(i+1)])
            print(row)
    def t3(self, n=1, a=False):
        p = self.offsets[n]
        row_size = self.row_size[n]
        size = self.row_count[n]
        if not a:
            size=30
        for i in range(size):
            row = buff_2_ints(self.data[p+row_size*i:p+row_size*(i+1)])
            print(row)
    def t4(self):
        p = self.offsets[3]
        row_size = self.row_size[3]
        size = self.row_count[3]
        for i in range(size):
            row = buff_2_int16s(self.data[p+row_size*i:p+row_size*(i+1)])
            print(row)            

In [475]:
temp = '''
        LatLng zz = new LatLng(%f, %f);
        zz = CoordTrans.gcjToBaidu(zz);
        MapStatus.Builder builder = new MapStatus.Builder();
        LatLng center = zz;
        float zoom = 19.0f; // 默认 11级
        builder.target(center).zoom(zoom);
        mMapView = new MapView(this, new BaiduMapOptions().mapStatus(builder.build()));
        initView(this);
        setContentView(layout);
        %s        
        MapView.setMapCustomEnable(false);
'''

def snippet_1(points):
    t2 = ''
    for x, y in points:
        t2 += 'mMapView.getMap().addOverlay(new DotOptions().center(CoordTrans.gcjToBaidu(new LatLng(%f, %f))).radius(6).color(0xFF0000FF));\n'%(y/100000, x/100000)
    return temp%(points[0][1]/100000, points[0][0]/100000, t2)

In [538]:
ppp = []
for i in range(ir.row_count[0]):
    name_idx, ps = ir.get_named_link_points(i)
    if name_idx == 4354:
        print(i)
        print(snippet_1(ps))
        ppp += ps

4

        LatLng zz = new LatLng(31.370030, 121.257420);
        zz = CoordTrans.gcjToBaidu(zz);
        MapStatus.Builder builder = new MapStatus.Builder();
        LatLng center = zz;
        float zoom = 19.0f; // 默认 11级
        builder.target(center).zoom(zoom);
        mMapView = new MapView(this, new BaiduMapOptions().mapStatus(builder.build()));
        initView(this);
        setContentView(layout);
        mMapView.getMap().addOverlay(new DotOptions().center(CoordTrans.gcjToBaidu(new LatLng(31.370030, 121.257420))).radius(6).color(0xFF0000FF));
mMapView.getMap().addOverlay(new DotOptions().center(CoordTrans.gcjToBaidu(new LatLng(31.369630, 121.257760))).radius(6).color(0xFF0000FF));
mMapView.getMap().addOverlay(new DotOptions().center(CoordTrans.gcjToBaidu(new LatLng(31.369330, 121.258000))).radius(6).color(0xFF0000FF));
        
        MapView.setMapCustomEnable(false);

14

        LatLng zz = new LatLng(31.372180, 121.255560);
        zz = CoordTrans.gcjToBaidu(zz);
     

In [539]:
name_idx, ps = ir.get_named_link_points(400)
print(snippet_1(ppp))
print(rp.names[name_idx], name_idx)


        LatLng zz = new LatLng(31.370030, 121.257420);
        zz = CoordTrans.gcjToBaidu(zz);
        MapStatus.Builder builder = new MapStatus.Builder();
        LatLng center = zz;
        float zoom = 19.0f; // 默认 11级
        builder.target(center).zoom(zoom);
        mMapView = new MapView(this, new BaiduMapOptions().mapStatus(builder.build()));
        initView(this);
        setContentView(layout);
        mMapView.getMap().addOverlay(new DotOptions().center(CoordTrans.gcjToBaidu(new LatLng(31.370030, 121.257420))).radius(6).color(0xFF0000FF));
mMapView.getMap().addOverlay(new DotOptions().center(CoordTrans.gcjToBaidu(new LatLng(31.369630, 121.257760))).radius(6).color(0xFF0000FF));
mMapView.getMap().addOverlay(new DotOptions().center(CoordTrans.gcjToBaidu(new LatLng(31.369330, 121.258000))).radius(6).color(0xFF0000FF));
mMapView.getMap().addOverlay(new DotOptions().center(CoordTrans.gcjToBaidu(new LatLng(31.372180, 121.255560))).radius(6).color(0xFF0000FF));
mMapView.getMap().

In [457]:
names[7872], 2739-2375

('路', 364)

In [520]:
ir.row_count, ir.row_size, ir.offsets

((2062, 1045, 1558, 5246, 3107, 694, 129, 711, 225, 100, 5, 15),
 (40, 16, 4, 4, 16, 4, 40, 20, 4, 8, 24, 24),
 (108,
  82588,
  99308,
  105540,
  126524,
  176236,
  179012,
  184172,
  198392,
  199292,
  200092,
  200212))

In [377]:
buff_2_ints(cc[:108])

[50,
 23857479,
 555,
 4857,
 2739,
 983443,
 12845382,
 95,
 14,
 1048616,
 262148,
 262160,
 1310760,
 524292,
 1572888,
 108,
 95108,
 100932,
 103152,
 122580,
 166404,
 168016,
 168616,
 175136,
 175920,
 176680,
 176680]

In [491]:
ir = InfoRegionBuffer(cc)
ir.t3(0)

[0, 4334, -2147483387, 306796130, 963117056, 240779272, 0, 364, 17, 8388608]
[1, 13386, -2080374523, 306787730, 838336512, -1065484266, 0, 365, 1, 2129940]
[2, 14139, -2147483387, 306796114, 958398464, 239599633, 0, 366, 17, 337641488]
[3, 14139, -2147483387, 306796114, 959184896, -833945593, 0, 367, 17, 142606336]
[4, 14139, -2147483387, 306792034, 1006632960, 251658248, 0, 368, 17, 34816]
[5, 3174, -2147483323, 306787514, 584056832, 11796492, 0, 369, 4, 32772]
[6, 13444, -2147483323, 306791846, 827064320, 5439505, 0, 370, 2, 136314896]
[7, 3174, -2147483323, 306787770, 268435456, 10, 0, 371, 4, 603979776]
[8, 13444, -2147483323, 306791846, 876085248, 17694736, 0, 372, 2, 513]
[9, 13337, -2147483387, 306796130, 1009516544, -821362672, 0, 373, 17, 16897]
[10, 1945, -2147483387, 306792018, 982777856, 245694480, 0, 374, 17, 4227076]
[11, 13386, -2147483323, 306787750, 333709312, -1057423332, 0, 375, 2, 164609]
[12, 3174, -2147483323, 306787750, 304873472, 9109522, 0, 376, 2, 2129940]
[13

In [468]:
ir = InfoRegionBuffer(cc)
ir.t4()

[8, 0]
[-8, 0]
[-8, 0]
[-8, 0]
[-8, 0]
[1, -2]
[0, -3]
[2, -4]
[3, -2]
[4, 0]
[5, 0]
[-3, -1]
[-10, -3]
[-3, -1]
[3, 1]
[5, 1]
[-3, -1]
[-10, -3]
[-3, -1]
[-5, -1]
[-3, -1]
[3, 1]
[10, 3]
[3, 1]
[3, 1]
[10, 3]
[3, 1]
[3, 1]
[5, 1]
[-8, -2]
[-8, -2]
[8, 2]
[8, 2]
[-8, -2]
[-8, -2]
[8, 2]
[8, 2]
[0, -11]
[2, 0]
[13, 2]
[2, 0]
[0, -11]
[2, 0]
[13, 2]
[2, 0]
[0, 9]
[0, -11]
[0, -11]
[2, 0]
[13, 2]
[2, 0]
[0, 9]
[0, 9]
[0, 9]
[-2, 0]
[-13, 0]
[-2, 0]
[0, 9]
[-2, 0]
[-13, 0]
[-2, 0]
[0, -11]
[0, 9]
[-2, 0]
[-13, 0]
[-2, 0]
[0, -11]
[2, 0]
[13, 2]
[2, 0]
[-2, 0]
[-13, 0]
[-2, 0]
[-2, 0]
[-13, 0]
[-2, 0]
[0, -11]
[2, 0]
[13, 2]
[2, 0]
[2, 0]
[13, 2]
[2, 0]
[0, 9]
[2, 0]
[13, 2]
[2, 0]
[0, 9]
[-2, 0]
[-13, 0]
[-2, 0]
[2, 0]
[13, 2]
[2, 0]
[0, 9]
[2, 0]
[13, 2]
[2, 0]
[-2, 0]
[-13, -1]
[-2, 0]
[-2, 0]
[-13, -1]
[-2, 0]
[2, 0]
[13, 1]
[2, 0]
[2, 0]
[13, 1]
[2, 0]
[-2, 0]
[-13, -1]
[-2, 0]
[-2, 0]
[-13, -1]
[-2, 0]
[2, 0]
[13, 1]
[2, 0]
[2, 0]
[13, 1]
[2, 0]
[1, -3]
[2, -8]
[1, -4]
[1, -3]
[2, -8]

[-2, 4]
[-95, 163]
[-120, 176]
[-2, 2]
[-1, 2]
[-1, 2]
[-141, 210]
[-2, 3]
[-53, 83]
[-9, 16]
[-8, 7]
[22, 11]
[133, 24]
[6, -15]
[-6, -3]
[-67, 166]
[15, 6]
[-6, -2]
[-20, 0]
[-12, 1]
[0, 13]
[1, -6]
[9, 4]
[19, -44]
[7, -13]
[0, 10]
[25, -18]
[11, -15]
[7, -23]
[33, 11]
[24, 7]
[-7, 14]
[12, -24]
[28, -78]
[11, -22]
[4, -7]
[0, 14]
[-1, 7]
[-4, 5]
[-10, 7]
[13, -18]
[3, -2]
[11, -21]
[-5, 17]
[3, 4]
[3, 3]
[37, 25]
[-10, 14]
[-15, 28]
[-7, 15]
[1, 13]
[-34, 1]
[-13, -7]
[-23, -23]
[-14, -5]
[-43, -6]
[-54, 2]
[-29, 66]
[-4, 11]
[-3, 9]
[-12, 1]
[-19, 24]
[-9, 2]
[-18, -5]
[30, -77]
[10, 7]
[-12, 21]
[-15, 25]
[-3, 4]
[14, 5]
[20, -43]
[99, -3]
[5, -17]
[-8, 0]
[2, -10]
[-14, 12]
[-8, 5]
[89, 34]
[21, 6]
[0, -14]
[-8, -2]
[7, 3]
[29, -46]
[21, -2]
[2, -6]
[22, 8]
[26, 10]
[-4, 2]
[1, -3]
[12, 5]
[19, 5]
[-20, 34]
[-14, 25]
[-2, -19]
[18, -23]
[10, -34]
[-79, -50]
[-9, 15]
[-69, 69]
[-4, 8]
[11, 12]
[65, 21]
[11, -17]
[2, 1]
[24, 11]
[39, 26]
[12, 6]
[-1, 3]
[-15, 36]
[31, 19]
[-5, 7]


[-3, 5]
[2, 14]
[11, 14]
[3, 9]
[-6, 6]
[58, 31]
[1, -20]
[13, 9]
[11, 2]
[13, 6]
[74, 31]
[15, 4]
[26, 9]
[7, 5]
[5, 8]
[33, 5]
[0, 14]
[-44, -19]
[5, 27]
[2, 13]
[-2, 3]
[-3, 0]
[-5, 1]
[-9, 3]
[57, -2]
[11, -3]
[1, -9]
[1, -7]
[3, -10]
[6, -19]
[-9, 10]
[-3, 2]
[-5, 6]
[-12, 11]
[-7, 6]
[60, 19]
[11, 2]
[17, -18]
[41, 11]
[3, -25]
[45, 0]
[11, 1]
[28, 4]
[40, 8]
[5, -23]
[-7, 7]
[15, -20]
[11, 4]
[0, 6]
[5, -5]
[134, -13]
[18, -3]
[90, 29]
[-1, -14]
[36, 12]
[23, 10]
[0, -6]
[2, -6]
[6, -8]
[11, -5]
[6, -2]
[1, 8]
[21, 9]
[5, 1]
[2, -1]
[12, 4]
[60, 24]
[-10, 10]
[-66, -17]
[7, 7]
[7, -1]
[4, -5]
[-16, -3]
[-2, -5]
[-3, -2]
[0, -7]
[3, -20]
[2, -2]
[3, -2]
[13, 3]
[8, -4]
[7, -3]
[-15, -7]
[-10, 22]
[46, 19]
[3, -9]
[32, 2]
[14, 1]
[5, 1]
[2, 2]
[-1, 8]
[-1, 17]
[4, 6]
[46, 9]
[3, 5]
[77, 33]
[-21, 0]
[-12, -1]
[-14, -8]
[-11, -8]
[-3, -2]
[0, -1]
[-2, -1]
[-1, -2]
[0, -1]
[-1, -2]
[-11, 31]
[-7, 9]
[-16, -7]
[-5, 17]
[8, -12]
[-90, -43]
[-7, -1]
[-45, -19]
[5, 2]
[10, 0]
[5, 0]
[5,

In [470]:
ir = InfoRegionBuffer(cc)
ir.t3(4, 0)

[12145499, 3125374, 1, 0]
[12145507, 3125374, 1, 1]
[12145507, 3125374, 1, 2]
[12145507, 3125374, 1, 3]
[12145507, 3125374, 1, 4]
[12146861, 3126395, 6, 5]
[12146002, 3126335, 3, 11]
[12146002, 3126335, 2, 14]
[12146002, 3126335, 3, 16]
[12146010, 3126337, 2, 19]
[12145986, 3126330, 3, 21]
[12145986, 3126330, 5, 24]
[12146747, 3125791, 1, 29]
[12146747, 3125791, 1, 30]
[12146739, 3125789, 1, 31]
[12146739, 3125789, 1, 32]
[12146747, 3125791, 1, 33]
[12146747, 3125791, 1, 34]
[12146739, 3125789, 1, 35]
[12146739, 3125789, 1, 36]
[12146365, 3125227, 4, 37]
[12146365, 3125227, 5, 41]
[12146365, 3125227, 1, 46]
[12146365, 3125227, 5, 47]
[12146382, 3125218, 1, 52]
[12146382, 3125218, 4, 53]
[12146382, 3125218, 5, 57]
[12146382, 3125218, 1, 62]
[12146382, 3125227, 7, 63]
[12146382, 3125227, 3, 70]


In [513]:
ccc = rp.get_info_region_buff(level=1, idx=9)
ir = InfoRegionBuffer(ccc)
ir.get_path_points(8)

[(12126170, 3140324),
 (12126161, 3140323),
 (12126163, 3140315),
 (12126172, 3140316)]

In [474]:
ir = InfoRegionBuffer(cc)
ir.get_path_points(8)

[(12146002, 3126335),
 (12145999, 3126334),
 (12145989, 3126331),
 (12145986, 3126330)]

In [463]:
cc = open('/tmp/region_buff.bin', 'rb').read()
d1 = buff_2_ints(cc[:27*4])
d2 = buff_2_int16s(cc[:27*4])
buff_2_ints(cc[76:80]), buff_2_int16s(cc[44:48]), d1, d2
# d2

([122580],
 [16, 4],
 [50,
  23857479,
  555,
  4857,
  2739,
  983443,
  12845382,
  95,
  14,
  1048616,
  262148,
  262160,
  1310760,
  524292,
  1572888,
  108,
  95108,
  100932,
  103152,
  122580,
  166404,
  168016,
  168616,
  175136,
  175920,
  176680,
  176680],
 [50,
  0,
  2375,
  364,
  555,
  0,
  4857,
  0,
  2739,
  0,
  403,
  15,
  326,
  196,
  95,
  0,
  14,
  0,
  40,
  16,
  4,
  4,
  16,
  4,
  40,
  20,
  4,
  8,
  24,
  24,
  108,
  0,
  29572,
  1,
  -30140,
  1,
  -27920,
  1,
  -8492,
  1,
  -30204,
  2,
  -28592,
  2,
  -27992,
  2,
  -21472,
  2,
  -20688,
  2,
  -19928,
  2,
  -19928,
  2])

In [386]:
d1[4], d2[18], struct.unpack('i', cc[72:76])

(2739, 40, (103152,))

In [361]:
struct.unpack('iiii', cc[122580:122580+16])

(12145499, 3125374, 1, 0)

In [382]:
cc.find(b'\x72\x00\x00\x00\x3b\x37\x00\x00\x06\x00')

4668